In [1]:
%run ../Python/paths.py

In [2]:
# import pandas as pd


from pandas import DataFrame

In [3]:
df_paths = DataFrame({'key': list(paths_sim.keys()), 'path': list(paths_sim.values())},)
print(len(df_paths))
# df_paths.head()

57


In [4]:
%%capture
parallel = False  # not working so well at the moment
if parallel:
    from dask import delayed, dataframe
    delay_noise = []
    delay_vort = []

df_noise = DataFrame(columns=pd_columns)
df_vort = DataFrame(columns=pd_columns)

for row in df_paths.values:
    key = row[0]
    path = row[1]
    keyparams = keyparams_from_path(path)
    s = delayed(pandas_from_path)(path, key, as_df=True) if parallel else \
            pandas_from_path(path, key)

    if keyparams[0] == 'noise':
        if parallel:
            delay_noise.append(s)
        else:
            df_noise = df_noise.append(s, ignore_index=True)
    else:
        if parallel:
            delay_vort.append(s)
        else:
            df_vort = df_vort.append(s, ignore_index=True)
    
if parallel:
    df_noise = dataframe.from_delayed(delay_noise, df_noise)
    df_vort = dataframe.from_delayed(delay_vort, df_vort)

In [5]:
df_vort.head()

,$n$,$c$,$\nu_8$,$f$,$\epsilon$,$\frac{k_d}{k_f}$,$F_f$,$Ro_f$,$Bu$,$\min h$,$\frac{\max |\bf u|}{c}$,$t_{stat}$,$t_{\max}$,short name
0,1920,20,1.111773e-12,7.539822,1.032053,0.5,0.055516,0.161795,4.0,0,0,94.85351,99.85633,vortex_grid_c20nh1920Bu4.0efr1.00e-02
1,1920,100,1.111773e-12,37.699112,1.072994,0.5,0.011248,0.032781,4.0,0,0,98.05531,99.85633,vortex_grid_c100nh1920Bu4.0efr1.00e-01
2,1920,400,1.111773e-12,0.000000,1.105353,0.0,0.002840,inf,inf,0,0,92.85488,99.85944,vortex_grid_c400nh1920Buinfefr1.00e+01
3,192,20,2.395242e-05,0.000000,0.963074,0.0,0.054250,inf,inf,0,0,42.50000,49.79167,vortex_grid_c20nh192Buinfefr1.00e+00
4,1920,100,1.111773e-12,37.699112,1.112852,0.5,0.011386,0.033182,4.0,0,0,99.45610,99.85633,vortex_grid_c100nh1920Bu4.0efr1.00e+01


In [6]:
df_noise.head()

,$n$,$c$,$\nu_8$,$f$,$\epsilon$,$\frac{k_d}{k_f}$,$F_f$,$Ro_f$,$Bu$,$\min h$,$\frac{\max |\bf u|}{c}$,$t_{stat}$,$t_{\max}$,short name
0,1920,20,9.676842e-13,15.079645,0.962507,1.000000,0.054240,0.079038,1.0,0,0,99.12911,99.83433,noise_c20nh1920Bu1.0
1,1920,400,9.687440e-13,0.000000,1.107435,0.000000,0.002842,inf,inf,0,0,118.04430,119.84550,noise_c400nh1920Buinf
2,3840,20,6.006974e-15,0.000000,1.084657,0.000000,0.056443,inf,inf,0,0,118.89480,119.89740,noise_c20nh3840Buinf
3,960,20,1.562292e-10,0.000000,0.988188,0.000000,0.054718,inf,inf,0,0,48.03906,49.86719,noise_c20nh960Buinf
4,1920,20,9.676842e-13,10.662919,0.856888,0.707107,0.052178,0.107529,2.0,0,0,99.53039,99.83117,noise_c20nh1920Bu2.0


In [14]:
%%capture
if parallel:
    df_vort.compute()
    df_noise.compute()

In [7]:
df_vort.sort_values(by=[r'$c$', r'$n$', r'$Bu$'])

,$n$,$c$,$\nu_8$,$f$,$\epsilon$,$\frac{k_d}{k_f}$,$F_f$,$Ro_f$,$Bu$,$\min h$,$\frac{\max |\bf u|}{c}$,$t_{stat}$,$t_{\max}$,short name
3,192,20,2.395242e-05,0.000000,0.963074,0.000000,0.054250,inf,inf,0,0,42.50000,49.79167,vortex_grid_c20nh192Buinfefr1.00e+00
10,192,20,3.287818e-05,0.000000,179.196505,0.000000,0.309713,inf,inf,0,0,49.77109,49.87218,vortex_grid_c20nh192Buinfefr1.90e+01
16,1920,20,1.111773e-12,3.371911,0.968860,0.223607,0.054359,0.354245,20.0,0,0,98.84252,99.84521,vortex_grid_c20nh1920Bu20.0efr1.00e+00
0,1920,20,1.111773e-12,7.539822,1.032053,0.500000,0.055516,0.161795,4.0,0,0,94.85351,99.85633,vortex_grid_c20nh1920Bu4.0efr1.00e-02
11,1920,20,1.111773e-12,0.000000,1.001318,0.000000,0.054959,inf,inf,0,0,48.78788,49.89213,vortex_grid_c20nh1920Buinfefr1.00e+00
15,1920,20,1.111773e-12,0.000000,1.084931,0.000000,0.056448,inf,inf,0,0,95.91953,99.83554,vortex_grid_c20nh1920Buinfefr1.00e+01
17,1920,20,1.111773e-12,0.000000,1.020823,0.000000,0.055314,inf,inf,0,0,47.32249,49.82900,vortex_grid_c20nh1920Buinfefr1.00e-02
23,1920,20,1.111773e-12,0.000000,1.072829,0.000000,0.056238,inf,inf,0,0,99.36064,99.86332,vortex_grid_c20nh1920Buinfefr1.00e-01
25,240,20,4.663114e-06,0.000000,0.992615,0.000000,0.054799,inf,inf,0,0,11.14583,20.62500,vortex_grid_c20nh240Buinfefr1.00e+00
9,960,20,1.792955e-10,0.000000,0.388949,0.000000,0.040100,inf,inf,0,0,49.78222,49.88249,vortex_grid_c20nh960Buinfefr1.00e+00


In [8]:
df_noise.sort_values(by=[r'$c$', r'$n$', r'$Bu$'])

,$n$,$c$,$\nu_8$,$f$,$\epsilon$,$\frac{k_d}{k_f}$,$F_f$,$Ro_f$,$Bu$,$\min h$,$\frac{\max |\bf u|}{c}$,$t_{stat}$,$t_{\max}$,short name
17,1920,10,9.676842e-13,0.000000,1.075719,0.000000,0.112576,inf,inf,0,0,49.637120,49.840500,noise_c10nh1920Buinf
6,3840,10,6.000402e-15,0.000000,1.118855,0.000000,0.114061,inf,inf,0,0,47.041910,49.857460,noise_c10nh3840Buinf
8,7680,10,3.724795e-17,0.000000,1.197475,0.000000,0.116672,inf,inf,0,0,16.840440,16.940660,noise_c10nh7680Buinf
9,960,10,1.560583e-10,0.000000,0.981543,0.000000,0.109190,inf,inf,0,0,49.059630,49.881210,noise_c10nh960Buinf
16,1920,20,9.676842e-13,21.325838,0.869648,1.414214,0.052436,0.054030,0.5,0,0,99.170310,99.873870,noise_c20nh1920Bu0.5
0,1920,20,9.676842e-13,15.079645,0.962507,1.000000,0.054240,0.079038,1.0,0,0,99.129110,99.834330,noise_c20nh1920Bu1.0
4,1920,20,9.676842e-13,10.662919,0.856888,0.707107,0.052178,0.107529,2.0,0,0,99.530390,99.831170,noise_c20nh1920Bu2.0
18,1920,20,9.676842e-13,7.539822,0.852898,0.500000,0.052097,0.151832,4.0,0,0,99.238300,99.840720,noise_c20nh1920Bu4.0
10,1920,20,9.687440e-13,0.000000,1.023984,0.000000,0.055371,inf,inf,0,0,119.380700,119.882000,noise_c20nh1920Buinf
2,3840,20,6.006974e-15,0.000000,1.084657,0.000000,0.056443,inf,inf,0,0,118.894800,119.897400,noise_c20nh3840Buinf
